## House Prices

* 주택 구매자에게 자신이 꿈꾸는 주택에 대해 설명하도록 요청하면 지하실 천장의 높이나 동서 철도와의 근접성으로 시작하지 않을 것입니다.
* 그러나 이 데이터 세트는 (그러한 특성이) 침실의 수나 흰색 울타리보다 가격 협상에 훨씬 더 많은 영향을 미친다는 것을 증명합니다.
* 아이오와주 에임스에 있는 주거용 주택의 (거의) 모든 측면을 설명하는 79개의 설명 변수가 있는 이 경쟁은 각 주택의 최종 가격을 예측하는 데 도전합니다.

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/5407/media/housesbanner.png"/>


* https://www.kaggle.com/c/house-prices-advanced-regression-techniques/

해당 경진대회는 아래 논문의 데이터셋으로 만들어졌습니다.
http://jse.amstat.org/v19n3/decock.pdf

## 라이브러리 로드

In [ ]:
# pandas, seaborn, numpy 를 불러옵니다.

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

## 데이터셋 가져오기

In [ ]:
import os, platform

base_path = "house"
file_name = "house-price.zip"

def file_exist_check(base_path):
    
    if not os.path.exists(base_path):
        os.makedirs(base_path)
        
    if os.path.exists(f"{base_path}/train.csv"):
        print(f"{os.getcwd()}/{base_path} 경로에 파일이 있음")
        return
    
    # colab
    if os.path.exists(f"{file_name}"):
        print(f"{base_path} 경로에 압축을 해제함")
        !unzip {file_name} -d {base_path}
        return
    else:
        print(f"""https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data 에서 다운로드 하고
              실습 경로 {os.getcwd()}/{base_path}에 옮겨주세요.""")
        return
    
file_exist_check(base_path) 

In [ ]:
# 컬럼이 모두 보이도록 설정합니다.
pd.options.display.max_columns = None

In [ ]:
# index_col="Id"로 train 데이터를 불러옵니다.
train = pd.read_csv("house/train.csv", index_col="Id")
print(train.shape)
train.head(2)

In [ ]:
# test 데이터셋을 불러오고 개수를 출력해 봅니다.
test = pd.read_csv("house/test.csv", index_col="Id")
print(test.shape)
test.head(2)

In [ ]:
# set을 활용해서 컬럼을 비교합니다.
set(train.columns) - set(test.columns)

### train과 test 하나의 데이터로 합치기

In [ ]:
# concat
# 장점 : 전처리를 한번만 해도 됩니다.
# 단점 : test에만 등장하는 데이터를 피처에 사용하면 안 되는 대회 정책이 있을 때 정책 위반이 될 수 있습니다.
# 대회 데이터이기 때문에 test를 미리 알 수 있지만 현실세계에서는 불가능한 방법입니다.
df = pd.concat([train, test])
df.shape

## EDA

### 정답값 label 따로보기

In [ ]:
# SalePrice 의 displot
# plotly 의 width 와 같은 의미로 그래프의 너비, 가로길이를 의미합니다.
# sns.histplot, sns.kdeplot 으로도 유사하게 시각화 할 수 있습니다.
sns.displot(data=df, x="SalePrice", aspect=5, rug=True, kind="kde")
plt.show()

### 왜도와 첨도


* 히스토그램의 분포를 추정해서 밀도추정곡선을 그리기도 합니다.

#### 왜도
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Negative_and_positive_skew_diagrams_%28English%29.svg/700px-Negative_and_positive_skew_diagrams_%28English%29.svg.png" width="600">



<img src="https://upload.wikimedia.org/wikipedia/commons/c/cc/Relationship_between_mean_and_median_under_different_skewness.png" width="600">


* 비대칭도(非對稱度, skewness) 또는 왜도(歪度)는 실수 값 확률 변수의 확률 분포 비대칭성을 나타내는 지표이다. 왜도의 값은 양수나 음수가 될 수 있으며 정의되지 않을 수도 있다. 
    * 왜도가 음수일 경우에는 확률밀도함수의 왼쪽 부분에 긴 꼬리를 가지며 중앙값을 포함한 자료가 오른쪽에 더 많이 분포해 있다. 
    * 왜도가 양수일 때는 확률밀도함수의 오른쪽 부분에 긴 꼬리를 가지며 자료가 왼쪽에 더 많이 분포해 있다는 것을 나타낸다. 
    * 평균과 중앙값이 같으면 왜도는 0이 된다.
    
* 출처 : [비대칭도 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EB%B9%84%EB%8C%80%EC%B9%AD%EB%8F%84)
    
#### 첨도

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/Standard_symmetric_pdfs.svg/1920px-Standard_symmetric_pdfs.svg.png" width="200">

* 첨도(尖度, 영어: kurtosis 커토시스)는 확률분포의 뾰족한 정도를 나타내는 척도이다. 관측치들이 어느 정도 집중적으로 중심에 몰려 있는가를 측정할 때 사용된다. 
    * 첨도값(K)이 3에 가까우면 산포도가 정규분포에 가깝다. 
    * 3보다 작을 경우에는(K<3) 정규분포보다 더 완만하게 납작한 분포로 판단할 수 있으며, 
    * 첨도값이 3보다 큰 양수이면(K>3) 산포는 정규분포보다 더 뾰족한 분포로 생각할 수 있다.

* 출처 : [첨도 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%B2%A8%EB%8F%84)



* [scipy.stats.kurtosis — SciPy Manual](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kurtosis.html)
* Fisher의 정의를 사용하면 결과에서 3.0을 빼서 정규 분포에 대해 0.0이 됩니다.
* True 이면 피셔의 정의가 사용됩니다(정상 ==> 0.0). False이면 Pearson의 정의가 사용됩니다(정상 ==> 3.0)

In [ ]:
# SalePrice 의 왜도와 첨도를 구합니다.
print("왜도(Skewness):", train["SalePrice"].skew())
print("첨도(Kurtosis):", train["SalePrice"].kurtosis())

In [ ]:
# SalePrice를 정규분포로 변환하고 왜도와 첨도를 구합니다.
# log 변환

train["SalePrice_log1p"] = np.log1p(train["SalePrice"])

In [ ]:
print("왜도(Skewness):", train["SalePrice_log1p"].skew())
print("첨도(Kurtosis):", train["SalePrice_log1p"].kurtosis())

In [ ]:
# SalePrice 를 log 변환 전후 kdeplot

train[["SalePrice", "SalePrice_log1p"]].hist(figsize=(12, 3), bins=100);

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 3))
sns.kdeplot(train["SalePrice"], fill=True, ax=axes[0])
sns.kdeplot(train["SalePrice_log1p"], fill=True, ax=axes[1])

In [ ]:
# 왜도와 첨도의 정확한 수치까지 알아야할 필요가 있나요?
# => 정확한 수치까지 모르더라도 시각화를 해보면 알 수 있습니다.
# 그런데 변수가 100개 그 이 상이라면? 물론 하나씩 다 비교해 볼 수 있습니다. 하지만 시간이 많이 필요합니다.
# 그래서 왜도와 첨도는 변수가 수백개 될 때 전체적으로 왜도와 첨도가 높은 값을 추출해서 전처리 할 수 있습니다.
# 보면 왜도나 첨도가 높은 값을 추출해서 변환하는 전처리를 진행할 수도 있습니다.
# pandas, numpy 등으로 기술통계를 구해보면 왜도, 첨도(기본 값 피셔의 정의 일 때) 0에 가까울 때 정규분포에 가까운 모습입니다.

### 결측치 보기

In [ ]:
# 결측치 확인
# 결측치 수와 비율을 함께 보고 싶다면 합계와 비율을 구해서 concat으로 합쳐줍니다.
# concat 에서 axis=0 은 행, axis=1은 열을 의미합니다.
# 메서드체이닝을 할 때는 도움말 보기가 잘 동작하지 않기 때문에 도움말을 보기 위해서는 변수에 할당하고 사용하는 것도 좋습니다.

isna_sum = df.isnull().sum()
isna_mean = df.isnull().mean()

pd.concat([isna_sum, isna_mean], axis=1).nlargest(10, 1)

In [ ]:
# 결측치가 많다고 삭제하는게 무조건 나은 방법이 아닐 수도 있습니다.
# 이상치, 특잇값을 찾는다면 오히려 특정 값이 신호가 될 수도 있습니다.
# 범주형 값이라면 결측치가 많더라도 채우지 않고 인코딩해주면 나머지 없는 값은 0으로 채워지게 됩니다.
# 그 대신 희소한 행렬이 생성됩니다.
# 수치데이터인데 결측치라면 잘못채웠을 때 오해할수 있으니 주의가 필요합니다.
# 언제 어떤 방법을 사용해야할지 모르겠다면 노트북 필사를 여러 개 해보고 다양한 데이터셋을 다뤄보는것을 추천합니다.
# 지금까지 다뤄본 데이터셋이 많지 않기 때문에 좀 더 수련이 필요합니다.
# 경험치를 많이 쌓는게 필요합니다.
null_feature = isna_mean[isna_mean > 0.8].index
df = df.drop(columns=null_feature)
df.shape

In [ ]:
# 어떤 피처가 삭제되었는지 확인하기 위해 
# train 컬럼으로 비교하게 되면 label값(SalePrice)까지 함께 비교하기 때문에 test 로 비교했습니다.
set(test.columns) - set(df.columns)

## 수치형 변수 
### 집값과 상관계수가 높은 데이터 보기

In [ ]:
corr = df.corr()

In [ ]:
# SalePrice와 상관계수가 특정 수치 이상인 데이터만 봅니다.
corr.loc[abs(corr["SalePrice"]) > 0.6, "SalePrice"]

In [ ]:
# SalePrice와 상관계수가 특정 수치 이상인 변수의 인덱스를 봅니다.
high_corr_col = corr.loc[abs(corr["SalePrice"]) > 0.5, "SalePrice"].index
high_corr_col

In [ ]:
# pairplot을 모든 변수에 대해 그리면 오래 걸립니다.
# 위에서 그렸던 상관계수가 높은 변수만 그려봅니다.
# 이상치, 상관계수, 왜도, 첨도 등을 한눈에 볼 수 있습니다.
# sns.pairplot(train[['OverallQual', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', '1stFlrSF',
#        'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'GarageCars', 'GarageArea',
#        'SalePrice']])
# sns.pairplot(train[high_corr_col], corner=True)

### 파생변수 만들기

In [ ]:
# TotalSF = TotalBsmtSF + 1stFlrSF + 2ndFlrSF
df["TotalSF"] = df["TotalBsmtSF"] + df["1stFlrSF"] + df["2ndFlrSF"]

### 결측치 채우기

* 임의의 값으로 채우기
* 평균, 중앙값으로 채우기
* 제거하기
* 머신러닝 기법으로 채우기
* [Imputing missing values before building an estimator — scikit-learn documentation](https://scikit-learn.org/stable/auto_examples/impute/plot_missing_values.html#sphx-glr-auto-examples-impute-plot-missing-values-py)

#### 0 이나 None 으로 채우기

In [ ]:
# Garage 관련 수치형 변수 0 으로 결측치 대체
Garage_0 = ['GarageYrBlt', 'GarageArea', 'GarageCars']
print(df[Garage_0].isnull().sum())
df[Garage_0] = df[Garage_0].fillna(0)
print(df[Garage_0].isnull().sum())

In [ ]:
# Garage 관련 범주형 변수 'None' 으로 결측치 대체
# 범주형 변수는 결측치를 채우지 않으면 따로 인코딩 하지 않습니다.
Garage_None = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
print(df[Garage_None].isnull().sum())
df[Garage_None] = df[Garage_None].fillna("None")

In [ ]:
df[Garage_None].isnull().sum()

In [ ]:
# Basement 관련 수치형 변수 0 으로 결측치 대체
Basement_0 = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 
              'BsmtFullBath', 'BsmtHalfBath']
print(df[Basement_0].isnull().sum())
df[Basement_0] = df[Basement_0].fillna(0)
print(df[Basement_0].isnull().sum())

In [ ]:
# Basement 관련 범주형 변수 'None' 으로 결측치 대체
Basement_None = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
print(df[Basement_None].isnull().sum())
df[Basement_None] = df[Basement_None].fillna("None")
print(df[Basement_None].isnull().sum())

#### 최빈값으로 채우기

In [ ]:
# 범주형 변수는 결측치를 채우지 않으면 따로 인코딩 하지 않습니다.
# 없는 값도 의미를 부여하고자 할 때는 채울 수 있습니다.
fill_mode = ['MSZoning', 'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType', 'Functional']
df[fill_mode].mode().loc[0]

In [ ]:
df[fill_mode].describe().loc["top"]

In [ ]:
print(df[fill_mode].isnull().sum())
df[fill_mode] = df[fill_mode].fillna(df[fill_mode].mode().loc[0])
print(df[fill_mode].isnull().sum())

In [ ]:
train[fill_mode[0]].unique()

In [ ]:
df[fill_mode[0]].value_counts()

In [ ]:
df[fill_mode[1]].value_counts()

### 데이터 타입 바꾸기

In [ ]:
# 수치 데이터의 nunique 구해서 어떤 값을 one-hot-encoding 하면 좋을지 찾아봅니다.
# 수치 데이터를 그대로 ordinal encoding 된 값을 그대로 사용해도 되지만 범주 값으로 구분하고자 하면
# category 나 object 타입으로 변환하면 one-hot-encoding 할 수 있습니다.
# ordinal encoding => one-hot-encoding 실습 목적
num_nunique = df.select_dtypes(include="number").nunique().sort_values()
num_nunique[num_nunique < 10]

In [ ]:
df["MoSold"].value_counts()

In [ ]:
# MSSubClass=The building class
# Changing OverallCond into a categorical variable
# Year and month sold are transformed into categorical features.
num_to_str_col = ["MSSubClass", "OverallCond", "YrSold", "MoSold"]
df[num_to_str_col].nunique()

In [ ]:
# 문자 형태로 변경하게 되면 나중에 pd.get_dummies 로 인코딩 했을 때 원핫인코딩을 합니다.
df[num_to_str_col] = df[num_to_str_col].astype(str)
df[num_to_str_col].dtypes

### 나머지 수치 변수 중앙값으로 결측치 대체

In [ ]:
# 수치형 변수 찾기
feature_num = df.select_dtypes(include="number").columns.tolist()
feature_num.remove("SalePrice")
feature_num

In [ ]:
# 수치형 변수 모두 중앙값으로 대체
# 수치형 변수를 대체할 때는 원래의 값이 너무 왜곡되지 않는지도 주의가 필요합니다.
# 중앙값(중간값), 평균값 등의 대표값으로 대체할 수도 있지만, 
# 회귀로 예측해서 채우는 방법도 있습니다.
df[feature_num] = df[feature_num].fillna(df[feature_num].median())
df[feature_num].isnull().sum().sum()

### 로그 변환

### 왜도 Skewness

비대칭도(非對稱度, skewness) 또는 왜도(歪度)는 실수 값 확률 변수의 확률 분포 비대칭성을 나타내는 지표이다. 왜도의 값은 양수나 음수가 될 수 있으며 정의되지 않을 수도 있다.

* 왜도가 음수일 경우에는 확률밀도함수의 왼쪽 부분에 긴 꼬리를 가지며 중앙값을 포함한 자료가 오른쪽에 더 많이 분포해 있다.
* 왜도가 양수일 때는 확률밀도함수의 오른쪽 부분에 긴 꼬리를 가지며 자료가 왼쪽에 더 많이 분포해 있다는 것을 나타낸다.
* 평균과 중앙값이 같으면 왜도는 0이 된다.


### 첨도
첨도(尖度, 영어: kurtosis 커토시스)는 확률분포의 뾰족한 정도를 나타내는 척도이다. 관측치들이 어느 정도 집중적으로 중심에 몰려 있는가를 측정할 때 사용된다.

첨도값(K)이 3에 가까우면 산포도가 정규분포에 가깝다.
3보다 작을 경우에는(K<3) 정규분포보다 더 완만하게 납작한 분포로 판단할 수 있으며,
첨도값이 3보다 큰 양수이면(K>3) 산포는 정규분포보다 더 뾰족한 분포로 생각할 수 있다.

* Fisher의 정의를 사용하면 결과에서 3.0을 빼서 정규 분포에 대해 0.0이 됩니다.
* True 이면 피셔의 정의가 사용됩니다(정상 ==> 0.0). False이면 Pearson의 정의가 사용됩니다(정상 ==> 3.0)


In [ ]:
train["SalePrice"].skew(), train["SalePrice"].kurt()

In [ ]:
train["SalePrice_log1p"].skew(), train["SalePrice_log1p"].kurt()

In [ ]:
# 왜도가 특정 수치 이상인 데이터를 따로 봅니다.
# 마이너스 값이 나올 수 있기 때문에 절대값을 씌우고 봅니다.
# 왜도가 큰 값을 skewed_col 변수에 담습니다.

feature_skew = abs(df.skew()).sort_values(ascending=False)

skewed_col = feature_skew[feature_skew > 2].index
skewed_col

In [ ]:
# 단순히 왜도만 보기 보다는 범주형 변수인지도 함께 봅니다.
# 수치 데이터인데 범주형태로 되어있는 데이터에 로그변환을 해줄 필요는 없습니다.
df[skewed_col].hist(bins=50, figsize=(12, 10));

### 로그 변환

In [ ]:
# 범주형태의 데이터를 제외하고 로그변환을 해주세요!
# 범주형태인지는 nunique() , select_dtypes() 등을 활용해 볼 수 있습니다.
log_features = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',
                 'TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea',
                 'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',
                 'TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF',
                 'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','YearRemodAdd']

In [ ]:
num_cate_feature = df[log_features].nunique()
num_cate_feature = num_cate_feature[num_cate_feature < 20].index
num_cate_feature

In [ ]:
num_log_feature = list(set(log_features) - set(num_cate_feature))
num_log_feature

In [ ]:
df[num_log_feature].hist(figsize=(12, 10));

In [ ]:
df[num_log_feature].describe()

In [ ]:
df[num_log_feature].dtypes

In [ ]:
# log 변환
# 같은 변수에 담아줄 때 로그 변환이 여러번 되지 않도록 셀 실행을 여러번 하지 않는 것에 유의합니다.
# 실수를 할 것 같다면 다른 변수에 담아주어도 됩니다.
df[num_log_feature] = np.log1p(df[num_log_feature])

In [ ]:
# histplot
# 그래프를 시각화 하면 0인 값이 많은데 0으로 채워도 되는 값이 있고 안 되는 값이 있습니다.
# 수치데이터의 결측치를 0으로 채우면 안 되는 값?

df[num_log_feature].hist(figsize=(12, 8), bins=100);

### squared features (Polynomials)

* 수학에서 다항식은 한 개 이상의 항의 합으로 이루어진 식이다. 즉, 단항식의 결합으로 이루어진 식이다. 예를 들어, x² - 2x + 3, 4x³, 5xy + 6은 모두 다항식이다. 다항식의 근과 다항식환 등은 대수학에서 중요하게 다루어진다.

* https://ko.wikipedia.org/wiki/%EB%8B%A4%ED%95%AD%EC%8B%9D

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/Terms-coefficient-ko.svg/700px-Terms-coefficient-ko.svg.png" width="300">

In [ ]:
df.columns

In [ ]:
# plt.show()는 그래프를 보여주는 역할을 합니다. 
# 기존 주피터 에서는 그래프를 보여주는게 기본값이 아니었어요.
# 그런데 마지막 줄에 그래프를 그리는 코드가 있다면 보여주는 것이 기본 값으로 변경이 되었습니다.
# 그래서 plt.show()를 했을 때 주피터 버전에 따라 중복 출력이 될 수도 있는데 이때는 plt.show()를 지워주시면 됩니다. 
squared_features = ['YearRemodAdd', 'LotFrontage', 
              'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea','GarageArea', 'TotalSF']
df[squared_features].hist(figsize=(12, 8), bins=50)
plt.show()

In [ ]:
# squared_features
# 사이킷런에서 Polynomials 로 변수를 만드는 기능을 제공하고는 있지만 직접 제곱을 해주어도 됩니다.
# 유니폼한 분포에 사용합니다. 여기에서는 사용할만한 변수가 보이지는 않지만 Polynomials 을 사용해 볼 수도 있다는 점을 다시 한번 생각해 보겠습니다.
df_square = df[squared_features] ** 2 

In [ ]:
# histplot
# uniform 한 분포가 아니기 때문에 여기에서는 사용하지 않아도 괜찮아 보입니다.
# 값을 강조해서 구분해서 보고자 할 때 주로 사용합니다.
df_square.hist(bins=100);

## 범주형 변수 보기

In [ ]:
# select_dtypes
df_cate = df.select_dtypes(include="object")
df_cate.head(2)

In [ ]:
# 범주형 변수에서 결측치가 없는 변수 선택
# 범주형 변수 중에 결측치가 있는지 확인을 해보고 어떤 범주형 변수를 선택해서 모델에 사용할지 의사결정을 하게 됩니다.
# 위에서 이미 결측치를 대부분 채워주었기 때문에 결측치가 거의 없지만 여전히 남아있는 결측치도 있습니다.
# 정렬을 하고 결측치가 있는 데이터를 제거하기 위해 슬라이싱을 사용하겠습니다.
# 범주형 데이터는 원핫인코딩 작업을 하기 때문에 결측치를 남겨두어도 상관은 없습니다.
# 없는 값은 변수로 생성하지 않기 때문입니다.
feature_cate = df_cate.isnull().mean().sort_values()[:-2].index
feature_cate

## make feature

In [ ]:
# label_name 변수에 예측에 사용할 정답 값 지정하기
# Submissions are evaluated on Root-Mean-Squared-Error (RMSE) 
# between the logarithm of the predicted value 
# and the logarithm of the observed sales price. 
# (Taking logs means that errors
# in predicting expensive houses and cheap houses will affect the result equally.)
# 1) 2억=>4억으로 예측하는 집값은, 2) 100억=>110억으로 예측했을 때 어디에 더 패널티를 줄 것인지
# MAE => 1) 2억차이, 2)10억, 오차의 절대값
# MSE => 1) 4억차이, 2)100억차이, 오차가 크면 클수록 값은 더 벌어집니다.
# RMSLE => 1) np.log(2) => 0.69, 2) np.log(10) => 2.30 
# => 로그값은 작은 값에서 더 패널티를 주고 값이 커짐에 따라 완만하게 증가합니다.
# => 로그값이 작은 값에서 더 패널티를 주는 것은 로그 그래프를 떠올려 보세요.

label_name = "SalePrice_log1p"
label_name

### feature_names

In [ ]:
df.columns

In [ ]:
# append 봉지과자를 통째로 넣음
a = []
a.append([1, 2, 3])
a

In [ ]:
# extend 봉지과자를 뜯어서 낱개로 넣음
a = []
a.extend([1, 2, 3])
a

In [ ]:
# 과자의 봉지는 iterable 혹은 컨테이너라고 부릅니다.
# list.append(x)는 리스트에 전달받은 요소(x)를 추가하는 메서드이며,
# list.extend(iterable)은 순환 가능한 요소(iterable)를 인자로 받으며, 
# 해당 컨테이너 안에 있는 모든 요소들을 리스트에 추가합니다.

In [ ]:
# 정답인 SalePrice는 꼭 제외해 주어야 합니다.
feature_names = []
feature_names.extend(num_log_feature)
feature_names.append("TotalSF")
feature_names.extend(feature_cate)
feature_names.remove("1stFlrSF")
feature_names.remove("2ndFlrSF")
feature_names.remove("BsmtFinSF1")
feature_names.remove("BsmtFinSF2")
# feature_names.remove("SalePrice")
feature_names

### One-Hot-Encoding

In [ ]:
df[feature_names].head(2)

In [ ]:
# 피처에 범주형 변수 개수 확인하기
# (2919, 41) 41 개의 범주형 변수가 있습니다.
df[feature_names].select_dtypes(include="O").shape

In [ ]:
df[feature_names].select_dtypes(include="O")

In [ ]:
# 원핫인코딩을 합니다.
# train, test 가 concat 되어 있는 상태라면 get_dummies 를 사용하는 것이 가장 간단합니다.
df_ohe = pd.get_dummies(df[feature_names])
df_ohe.head(2)

In [ ]:
# 원핫인코딩 후 기존 데이터와 개수를 비교합니다.
df[feature_names].shape, df_ohe.shape

## 데이터셋 만들기
* 학습과 예측에 사용할 데이터셋 만들기

In [ ]:
df_ohe.head(2)

In [ ]:
# train, test 데이터셋 나누기
# train, test 의 인덱스 값을 확인해서 나누기에 적절한지 확인해 보기 위해 출력해 봤습니다.
train.index, test.index

In [ ]:
# 데이터셋을 나눕니다.
X_train = df_ohe.loc[train.index]
X_test = df_ohe.loc[test.index]

print(X_train.shape, X_test.shape)
print(X_train.isnull().sum().sum(), X_test.isnull().sum().sum())

In [ ]:
# label_name 확인
label_name

In [ ]:
# y_train 만들기 - 학습에 사용할 정답값 예) 기출문제의 정답
y_train = train["SalePrice_log1p"]
print(y_train.shape)
y_train.head(2)

## 머신러닝 모델

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42)
model

### KFold 로 Cross Validation 하기

In [ ]:
# KFold 를 사용해서 분할을 나눕니다. 
# 분할에 random_state를 사용할 수 있습니다.
# 분할 때문에 값이 변경된건지 실험을 좀 더 고정할 수 있습니다.
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf

### cross_val_score 와 cross_val_predict 로 점수 구하기

In [ ]:
# cross_val_predict 로 예측 결과를 구합니다.
# y_valid_predict 에 결과를 담습니다.
from sklearn.model_selection import cross_val_predict

y_valid_predict = cross_val_predict(model, X_train, y_train, cv=kf, n_jobs=-1)
y_valid_predict

### Metric

In [ ]:
# RMSE => 원래는 RMSLE 인데 로그를 이미 적용해 주어서 RMSE로 계산합니다.
mse = np.square(y_train - y_valid_predict).mean()
rmse = np.sqrt(mse)
rmse

### 실제값과 예측값 비교하기

In [ ]:
# regplot 으로 예측값에 대한 회귀선 그리기
sns.regplot(x=y_train, y=y_valid_predict)

In [ ]:
# r^2 score
# 다른 오차측정 방법은 작을 수록 오차가 적음을 의미하지만
# r^2 score 는 1에 가까운 큰 값일 수록 잘 예측한 값입니다.
from sklearn.metrics import r2_score

r2_score(y_train, y_valid_predict)

In [ ]:
# kdeplot으로 실제값과 예측값 비교하기
sns.kdeplot(y_train)
sns.kdeplot(y_valid_predict)

In [ ]:
# 여기까지 모의고사를 풀어보고 점수가 어느정도 나오는지를 확인했습니다.

## 학습(훈련)과 예측

In [ ]:
# 학습하고 예측하기
# 모의고사를 풀어보고 바로 실전을 풀어봅니다.
y_predict = model.fit(X_train, y_train).predict(X_test)
y_predict

### 피처 중요도 보기

In [ ]:
# 중요도 상위 피처만 가져오기

fi = pd.Series(model.feature_importances_)
fi.index = model.feature_names_in_
fi.nlargest(20)

In [ ]:
# 피처 중요도 시각화 하기
fi.nlargest(20).plot.barh()

## Submit

* https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/submit

In [ ]:
# 제출 파일 양식을 불러옵니다.
submit = pd.read_csv("house/sample_submission.csv")
submit.head(2)

In [ ]:
# 정답을 옮겨 적습니다.
# 리더보드에 있는 점수와 동일한 스케일 점수를 미리 계산해 보기 위해서는 로그 적용한 값으로 계산해 주지만
# 제출할 때는 지수함수를 적용해서 원래 스케일로 복원하여 제출합니다.
# 주의! 내부에서 평가할 때는 제출받은 값에 로그를 취해서 점수를 평가하는데 
# 제출할 때는 지수함수를 적용해서 제출해야 합니다.
submit["SalePrice"] = np.expm1(y_predict)

In [ ]:
file_name = f"house/submission_{rmse:.5f}.csv"
file_name

In [ ]:
# csv 파일로 저장합니다.
submit.to_csv(file_name, index=False)

In [ ]:
# 제대로 저장되었는지 확인합니다.
pd.read_csv(file_name)

* 리더보드 상위권 노트북도 참고하는 것을 추천합니다. https://www.kaggle.com/code/serigne/stacked-regressions-top-4-on-leaderboard

## 수치형
* 결측치 대체(Imputation)
    * 수치형 변수를 대체할 때는 원래의 값이 너무 왜곡되지 않는지도 주의가 필요합니다.
    * 중앙값(중간값), 평균값 등의 대표값으로 대체할 수도 있지만, 
    * 회귀로 예측해서 채우는 방법도 있습니다.
* 스케일링 - Standard, Min-Max, Robust
* 변환 - log
* 이상치(너무 크거나 작은 범위를 벗어나는 값) 제거 혹은 대체
* 오류값(잘못된 값) 제거 혹은 대체
* 이산화 - cut, qcut

## 범주형
* 결측치 대체(Imputation)
* 인코딩 - label, ordinal, one-hot-encoding
* 범주 중에 빈도가 적은 값은 대체하기